In [ ]:
from ipynb.fs.full.boundaryCellTraction import *

In [3]:
class boundaryPointTraction(A):

    def __init__(self, edges, xy):    # The order matters for edges, bottom left corner on the bottom is : ["b", "l"]
        
        sw_S, sw_N, sw_W, sw_E = 0, 0, 0, 0

        if edges[0] == "b": sw_S = 1
        if edges[0] == "t": sw_N = 1
        if edges[0] == "l": sw_W = 1
        if edges[0] == "r": sw_E = 1
        
        a_P = (A(xy).a_N*sw_N + A(xy).a_S*sw_S )*2/(Sfy) + (A(xy).a_E*sw_E + A(xy).a_W*sw_W)*2/(Sfx)
        
        a_N = A(xy).a_N*2/(Sfy) # are terms cancel in derivation
        a_S = A(xy).a_S*2/(Sfy)
        a_E = A(xy).a_E*2/(Sfx)
        a_W = A(xy).a_W*2/(Sfx)

        if edges[0] == "b":
            self.a_N = a_P
            self.a_P = a_S
        if edges[0] == "t":
            self.a_S = a_P
            self.a_P = a_N
        if edges[0] == "l":
            self.a_E = a_P
            self.a_P = a_W
        if edges[0] == "r":
            self.a_W = a_P
            self.a_P = a_E
    
    def b_diff(edges, k, xy, U_previous, time):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        def N_term():
            
            N_term =( A.coef(xy, "N", uv)*(
                    (boundaryPointTraction.corner(edges, "E", uv, U_previous, k) - boundaryPointTraction.corner(edges, "W", uv, U_previous, k))
                    /dx)
            )

            return N_term

        def S_term():
            
            S_term =(
                    A.coef(xy, "S", uv)*(
                        (boundaryPointTraction.corner(edges, "E", uv, U_previous, k) - boundaryPointTraction.corner(edges, "W", uv, U_previous, k))
                        /dx)
                ) 
            return S_term
        def E_term():
            
            E_term =(
                    A.coef(xy, "E", uv)*(
                        (boundaryPointTraction.corner(edges, "N", uv, U_previous, k) - boundaryPointTraction.corner(edges, "S", uv, U_previous, k))
                        /dy)
                ) 
            return E_term
        def W_term():
            
            W_term =(
                        A.coef(xy, "W", uv)*(
                        (boundaryPointTraction.corner(edges, "N", uv, U_previous, k) - boundaryPointTraction.corner(edges, "S", uv, U_previous, k))
                        /dy)
                )
                
            return W_term

        # This can be tidied up/shortened

        if (edges[0] == "b") & (xy == "x") : b_diffusion =  tr_bottom_x(time)  + N_term()
        if (edges[0] == "b") & (xy == "y") : b_diffusion =  tr_bottom_y(time)  + N_term()
        if (edges[0] == "t") & (xy == "x") : b_diffusion =  tr_top_x(time) + S_term()
        if (edges[0] == "t") & (xy == "y") : b_diffusion =  tr_top_y(time)+ S_term()
        if (edges[0] == "l") & (xy == "x") : b_diffusion =  tr_left_x + E_term()
        if (edges[0] == "l") & (xy == "y") : b_diffusion =  tr_left_y + E_term()
        if (edges[0] == "r") & (xy == "x") : b_diffusion =  tr_right_x(time) + W_term()
        if (edges[0] == "r") & (xy == "y") : b_diffusion =  tr_right_y(time) + W_term()  

        return b_diffusion
    
    def corner(edges, corner_placement, uv, U_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)


        if corner_placement == "E":
            corner =  (1/2)*(disp.E + disp.P)
        if corner_placement == "W":
            corner =  (1/2)*(disp.W + disp.P)
        if  corner_placement == "N":
            corner =  (1/2)*(disp.N + disp.P)
        if  corner_placement == "S":
            corner =  (1/2)*(disp.S + disp.P)

        if len(edges) > 1:

            if (edges[1] == "r") & (corner_placement == "E"):
                    corner =  disp.E
            if (edges[1] == "l") & (corner_placement == "W"):
                    corner =  disp.W
            if (edges[1] == "t") & (corner_placement == "N"):
                    corner =  disp.N
            if (edges[1] == "b") & (corner_placement == "S"):
                    corner =  disp.S
        
        return corner
# boundaryCellTraction.b_diff(["b", "l"], 14, "x", U_previous)
   

In [ ]:
def traction_point_BCs_A(A_matrix, k, edges, xy, time):
    
    #ap
    A_matrix[k,k] = boundaryPointTraction(edges, xy).a_P

    if edges[0] == "b":
        #an
        A_matrix[k,index(k).n] = - boundaryPointTraction(edges, xy).a_N
    if edges[0] == "t":
        #as
        A_matrix[k,index(k).s] = - boundaryPointTraction(edges, xy).a_S
    if edges[0] == "l":
        #ae
        A_matrix[k, index(k).e] = - boundaryPointTraction(edges, xy).a_E
    if edges[0] == "r":
        #aw
        A_matrix[k, index(k).w] = -  boundaryPointTraction(edges, xy).a_W

    return A_matrix

def traction_point_BCs_b(b_matrix, k, edges, xy, U_previous, time):
    
    #b
    b_matrix[k] =boundaryPointTraction.b_diff(edges, k, xy, U_previous, time)

    return b_matrix
